# Regelbasierte OCR-Nachbearbeitung

Let's list some typical regular errors in our corpus:

* "fie" statt "sie" (add pic & result later)
* "vm", "vnd" statt "um", "und"
* "<" statt "ch"

Some things are not errors, but rather features of historical orthograpy which we would like to normalise for further processing with modern NLP tools:

* "ſ" statt "s"

In many cases we can fix it with some regular search-and replace patterns (e.g. take each `<` not surrounded by spaces and convert into `ch`)

The standard way to express & implement such patterns on a computer would be regular expressions. You can learn more about regular expressions [here](https://www.w3schools.com/python/python_regex.asp).

## Implementing rules for typical errors with regular expressions

In [2]:
import re

In [3]:
def post_correct_text(ocr_output):
    cleaner_output = re.sub(r'(\w)<(\w)', '\\1ch\\2', ocr_output)
    cleaner_output = re.sub(r'(\w)5(\w)', '\\1s\\2', cleaner_output)
    cleaner_output = re.sub(r'\bv(m|nd)\b', 'u\\1', cleaner_output)
    cleaner_output = re.sub(r'\bfie\b', 'sie', cleaner_output)
    cleaner_output = cleaner_output.replace('ſ','s')
    cleaner_output = cleaner_output.replace('\n',' ')
    return cleaner_output

## Applying rules to OCR results

In [4]:
import pytesseract
from PIL import Image
from aux.measure_ocr_quality import measure_ocr_quality

<img src="grippe.jpeg" width=700>

In [5]:
ocr_output = pytesseract.image_to_string(Image.open('grippe.jpeg'), lang='frk')

In [6]:
print(ocr_output)

Zie Grippe wüfel weiter

Zunahme der f<weren Fälle in Berlin.

Die Zahl der Grippefälle iſt in den leßten
beider Tagen auch in Groß-Berlin noH
erf>liS zeitiegen. Die Worenhäuſer und ſon-
Haen aroßen GeſhHäfte, die Krirgs- unh die prie
n Betriebe lagen, daß übermäig viele An«

* fich hcben rex? melden müſſen,-und an<
; “ Loft und 5ei der Straßenbahn iſt der
ſoz der Grippelranken bedeuten) g&

MeB R 2 8 1



In [7]:
ocr_output_corr = post_correct_text(ocr_output)

Let us see how the whole thing changed: 

In [8]:
print(ocr_output_corr)

Zie Grippe wüfel weiter  Zunahme der fchweren Fälle in Berlin.  Die Zahl der Grippefälle ist in den leßten beider Tagen auch in Groß-Berlin noH erf>liS zeitiegen. Die Worenhäuser und son- Haen aroßen GeshHäfte, die Krirgs- unh die prie n Betriebe lagen, daß übermäig viele An«  * fich hcben rex? melden müssen,-und an< ; “ Loft und 5ei der Straßenbahn ist der soz der Grippelranken bedeuten) g&  MeB R 2 8 1 


## Measuring the improvement

Let us see hot that affected the OCR quality metrics

In [9]:
ground_truth = 'Die Grippe wütet weiter. Zunahme der schweren Fälle in Berlin. Die Zahl der Grippefälle ist in den letzten Tagen auch in Groß-Berlin noch erheblich gestiegen. Die Warenhäuser und sonstigen großen Geschäfte, die Kriegs- und die privaten Betriebe klagen, daß übermäßig viele Angestellte sich haben krank melden müssen und auch bei der Post und bei der Straßenbahn ist der Prozentsatz der Grippekranken deutlich gestiegen.'

### Original (uncorrected) OCR result

In [12]:
precision, recall, f_score = measure_ocr_quality(ocr_output, ground_truth)

In [13]:
print(f'Precision: {round(precision, 4)}\nRecall: {round(recall, 4)}\nF1-score: {round(f_score, 4)}')

Precision: 0.7757
Recall: 0.7985
F1-score: 0.7869


### Corrected OCR result

In [14]:
precision, recall, f_score = measure_ocr_quality(ocr_output_corr, ground_truth)

In [15]:
print(f'Precision: {round(precision, 4)}\nRecall: {round(recall, 4)}\nF1-score: {round(f_score, 4)}')

Precision: 0.8091
Recall: 0.8309
F1-score: 0.8198


So, our F-measure increased a bit, good!

## (Advanced) Running on the whole corpus

In [16]:
from pathlib import Path
from tqdm import tqdm

In [17]:
pathtxt = Path('../data/txt')

for file in tqdm(pathtxt.iterdir()):
    if file.suffix == '.txt':
        text = file.read_text()
        corrected = post_correct_text(text)
        file.write_text(corrected)

103it [00:00, 110.62it/s]
